In [80]:
import os 

# data manipulation
import numpy as np
import pandas as pd

# geospatial
import geopandas as gpd
from gerrychain import Graph
import networkx as nx
from networkx import is_connected

pd.options.mode.chained_assignment = None

## Data

In [81]:
state_dict = {
    '01': 'Alabama',
    '02': 'Alaska',
    '04': 'Arizona',
    '05': 'Arkansas',
    '06': 'California',
    '08': 'Colorado',
    '09': 'Connecticut',
    '10': 'Delaware',
    '11': 'District_of_Columbia',
    '12': 'Florida',
    '13': 'Georgia',
    '15': 'Hawaii',
    '16': 'Idaho',
    '17': 'Illinois',
    '18': 'Indiana',
    '19': 'Iowa',
    '20': 'Kansas',
    '21': 'Kentucky',
    '22': 'Louisiana',
    '23': 'Maine',
    '24': 'Maryland',
    '25': 'Massachusetts',
    '26': 'Michigan',
    '27': 'Minnesota',
    '28': 'Mississippi',
    '29': 'Missouri',
    '30': 'Montana',
    '31': 'Nebraska',
    '32': 'Nevada',
    '33': 'New_Hampshire',
    '34': 'New_Jersey',
    '35': 'New_Mexico',
    '36': 'New_York',
    '37': 'North_Carolina',
    '38': 'North_Dakota',
    '39': 'Ohio',
    '40': 'Oklahoma',
    '41': 'Oregon',
    '42': 'Pennsylvania',
    '44': 'Rhode_Island',
    '45': 'South_Carolina',
    '46': 'South_Dakota',
    '47': 'Tennessee',
    '48': 'Texas',
    '49': 'Utah',
    '50': 'Vermont',
    '51': 'Virginia',
    '53': 'Washington',
    '54': 'West_Virginia',
    '55': 'Wisconsin',
    '56': 'Wyoming'
}

###### Voting Data ######
# From: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ
    # county_fips column used to join election data to graph. Format is {state_fips}{county_fips}
    # We Drop NAN values. These are state write in ballots for Conneticut and Rhode Island. They account for less than 10,000 votes. 
    # We do not consider alaska because the county FIPS codes correspond to different districts than census population tracts
        # Note: County results in Alaska for 2004 are based on official Alaska data, but it is clear the district returns significantly overstate the number of votes cast. In Alaska, the county_fips field stores a combination of state FIPS code and district.


election_df = pd.read_csv('../dataIn/elections/countypres_2000-2024.csv')
election_df.dropna(subset = ['county_fips'], inplace=True)
election_df['county_fips'] = election_df['county_fips'].astype(str)


###### BVAP Data ######
# src: https://www.census.gov/programs-surveys/decennial-census/about/voting-rights/cvap.2020.html#list-tab-1518558936
    
    # Data joined using last 5 digits of geiod {STATEFIPS}{COUNTYFIPS} 
    # Columns Used:
        # cvap_est:      The rounded estimate of the total number of United States citizens 18 years of age or older for that geographic area or group.
        # lnnumber:      The line number of the record.
            # 1: Total Population
            # 5: Black or African American alone
            # 7: White Alone
            # 13: Hispanic or Latino Alone
            
acs_df = pd.read_csv('../dataIn/elections/CVAP_ACS_2016-2020_county.csv', encoding = 'latin-1')
acs_df['COUNTYFP'] = acs_df['geoid'].str[-5:].astype(str)


###### Tigerline Shapefiles ######
# src: https://www.nhgis.org/gis-files
    # used to generate Json graph using gerrychain
gdf = gpd.read_file("../dataIn/elections/shapefiles/US_county_2020.shp")


### Generate Graphs and Load Data

In [86]:
###### Out Dir ###### 
outDir = "../dataIn/elections/state_graphs_2020/"
os.makedirs(outDir, exist_ok = True)

elections = [2000, 2004, 2008, 2012, 2016, 2020]

###### Graph Generation ######
## Iterate through each state
for stateFP in state_dict.keys():
    print('-' * 50)
    print(f"Constructing graph for: {state_dict[stateFP]}")
    # Remove States:
        # Alaska removed because the Harvard Election Dataset does not have county-level results
        # Hawaii removed because we restrict analysis to continental U.S. 
        # DC removed because it only has one node 
    if stateFP in ['02', '15', '11']:
        continue

    # filter out state shapefile
    state_gdf = gdf[gdf['STATEFP'] == stateFP]
    # Generate graph using Gerrychain
    G = Graph.from_geodataframe(state_gdf, adjacency = 'queen', ignore_errors = True)


###### Add BVAP, DEM%, and REP% Graph Nodes ######
    ## Iterate through nodes
    for node in G.nodes:

        # FIPS Code from graph node
        countyFIPS = ('{}{}'.format(G.nodes[node]['STATEFP'], G.nodes[node]['COUNTYFP']))
        # Filter election data by county
        election_slice = election_df[election_df['county_fips'] == countyFIPS]
        # Filter ACS data by county
        acs_slice = acs_df[acs_df['COUNTYFP'] == countyFIPS].set_index('lnnumber')

        ## Iterate through Elections
        for election in elections:
            
            ###### Add Voting Data ######
            try:
                total_votes = election_slice[election_slice['year'] == election]['totalvotes'].iloc[0]
                G.nodes[node][f'{election}_V'] = total_votes
                # DEM%
                G.nodes[node][f'{election}_D'] = election_slice[(election_slice['party'] == 'DEMOCRAT') & (election_slice['year'] == election)]['candidatevotes'].sum(skipna = True, numeric_only = True) / total_votes
                # REP%
                G.nodes[node][f'{election}_R'] = election_slice[(election_slice['party'] == 'REPUBLICAN') & (election_slice['year'] == election)]['candidatevotes'].sum(skipna = True, numeric_only = True) / total_votes
                # No Fill Metadata
                G.nodes[node][f'{election}_filled_from'] = {
                    "filled": False
                }
            # Assign highest DEM% of neighbors if no data is available for that node
            except IndexError:
                # init dem% = 0 and best neighbor
                G.nodes[node][f'{election}_D'] = 0 
                best_neighbor = None

                ## Iterate neighbors
                for neighbor in G.neighbors(node):
                    try:
                        neighbor_dem = G.nodes[neighbor][f'{election}_D']

                        # if neighbor value > current max, set as new max and record neighbor
                        if neighbor_dem > G.nodes[node][f'{election}_D']:
                            best_neighbor = neighbor
                            G.nodes[node][f'{election}_D'] = neighbor_dem

                    except KeyError:
                        pass

                # Node gets best neighbor's data
                print("-" * 10)
                print("County election data missing for: ")
                print(f"\t State            : {state_dict[stateFP]}")
                print(f"\t County FIPS      : {countyFIPS}")
                print(f"\t Election         : {election}")
                print(f"Filling with max value from neighboring nodes:")
                print(f"\t County FIPS      : {G.nodes[best_neighbor]['COUNTYFP']}{G.nodes[best_neighbor]['STATEFP']}")
                print(f"\t Fill Value Dem%  : {G.nodes[best_neighbor][f'{election}_D']}")
                print(f"\t Fill Value Rep%  : {G.nodes[best_neighbor][f'{election}_R']}")
                print(f"\t Fill Value Votes : {G.nodes[best_neighbor][f'{election}_V']}")
                print("-" * 10)

                G.nodes[node][f'{election}_D'] = G.nodes[best_neighbor][f'{election}_D']      
                G.nodes[node][f'{election}_R'] = G.nodes[best_neighbor][f'{election}_R']
                G.nodes[node][f'{election}_V'] = G.nodes[best_neighbor][f'{election}_V']

                # Fill Metadata
                G.nodes[node][f'{election}_filled_from'] = {
                "filled": True,
                "used_neighbor": best_neighbor,
                "values_used": 
                    {
                    "D": G.nodes[best_neighbor][f'{election}_D'],
                    "R": G.nodes[best_neighbor][f'{election}_R'],
                    "V": G.nodes[best_neighbor][f'{election}_V']
                    }
                }

            ###### Add ACS Data ######
            total_vap = acs_slice.loc[1]['cvap_est']
            G.nodes[node]['VAP'] = total_vap
            G.nodes[node]['BVAP'] = acs_slice.loc[5]['cvap_est'] / total_vap
            G.nodes[node]['HVAP'] = acs_slice.loc[13]['cvap_est'] / total_vap

    ###### Save Graph ######
    out_file = os.path.join(outDir, f'{state_dict[stateFP]}.json')
    G.to_json(out_file)
    print(f'{state_dict[stateFP]} graph completed and saved to: "{out_file}"', sep = '\t')
    print('-' * 50)

--------------------------------------------------
Constructing graph for: Alabama


/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CBSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column METDIVFP!
  warnings.warn("NA values found in column {}!".format(column))


Alabama graph completed and saved to: "../dataIn/elections/state_graphs_2020/Alabama.json"
--------------------------------------------------
--------------------------------------------------
Constructing graph for: Alaska
--------------------------------------------------
Constructing graph for: Arizona


/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CBSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column METDIVFP!
  warnings.warn("NA values found in column {}!".format(column))


Arizona graph completed and saved to: "../dataIn/elections/state_graphs_2020/Arizona.json"
--------------------------------------------------
--------------------------------------------------
Constructing graph for: Arkansas


...

West_Virginia graph completed and saved to: "../dataIn/elections/state_graphs_2020/West_Virginia.json"
--------------------------------------------------
--------------------------------------------------
Constructing graph for: Wisconsin


/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CBSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column METDIVFP!
  warnings.warn("NA values found in column {}!".format(column))


Wisconsin graph completed and saved to: "../dataIn/elections/state_graphs_2020/Wisconsin.json"
--------------------------------------------------
--------------------------------------------------
Constructing graph for: Wyoming


/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column CBSAFP!
  warnings.warn("NA values found in column {}!".format(column))
/Users/dgottschalk/miniconda3/envs/uncg/lib/python3.10/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column METDIVFP!
  warnings.warn("NA values found in column {}!".format(column))


Wyoming graph completed and saved to: "../dataIn/elections/state_graphs_2020/Wyoming.json"
--------------------------------------------------
